<a href="https://colab.research.google.com/github/arsenitheunicorn/nlp4thgrade/blob/main/bert_project/bert_gender_assuming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('/content/gender_project/df_modern.csv', encoding='utf-8')
df['text_len'] = df['lyrics'].apply(lambda x: str(x).count(" ") + 1)

In [3]:
df

,lyrics,gender,text_len
0,wouldn angry fuck tear arouse refresh supply s...,1,73
1,stay super late tonight pick apples make pies ...,1,50
2,girl come think look cool sing say wanna like ...,1,105
3,problem want follow underneath insane asylum w...,1,46
4,little dive deadend road call crosseyed cricke...,1,107
...,...,...,...
2469,simply have wonderful christmastime simply hav...,0,32
2470,ching ching ching ching ching money spend toda...,0,99
2471,dirty south uhhuh feel feel east coast feel fe...,0,81
2472,passion instant sweat bead feel cupid shoot he...,0,33


In [4]:
df['text_len'].mean(), df['text_len'].median()

(130.42724333063865, 87.0)

In [5]:
df[df["text_len"] > 256].groupby('gender').count()

,lyrics,text_len
gender,,
0,119,119
1,174,174


In [6]:
sentences = df.lyrics.values
labels = df.gender.values

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
print('Original: ', sentences[0])
print('Tokenized: ', tokenizer.tokenize(sentences[0]))
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(sentences[0])))

Original:  wouldn angry fuck tear arouse refresh supply suffer fade scream repulsion invade goodbye dig deep inside get past agony away rot vain wouldn angry things stay fight torment help fee pain suffer fade scream repulsion invade goodbye dig deep inside get past agony away rot hear scat sing fuck scat sing scat sing hate scat sing dig deep inside get past agony away rot dig deep inside get past agony away rot vain
Tokenized:  ['wouldn', 'angry', 'fuck', 'tear', 'ar', '##ouse', 'ref', '##resh', 'supply', 'suffer', 'fade', 'scream', 'rep', '##ulsion', 'invade', 'goodbye', 'dig', 'deep', 'inside', 'get', 'past', 'agony', 'away', 'rot', 'vain', 'wouldn', 'angry', 'things', 'stay', 'fight', 'torment', 'help', 'fee', 'pain', 'suffer', 'fade', 'scream', 'rep', '##ulsion', 'invade', 'goodbye', 'dig', 'deep', 'inside', 'get', 'past', 'agony', 'away', 'rot', 'hear', 'sc', '##at', 'sing', 'fuck', 'sc', '##at', 'sing', 'sc', '##at', 'sing', 'hate', 'sc', '##at', 'sing', 'dig', 'deep', 'inside'

In [10]:
input_ids = []
for sent in sentences:
    encoded_sent = tokenizer.encode(
                        str(sent),                      
                        add_special_tokens = True)
    input_ids.append(encoded_sent)

Token indices sequence length is longer than the specified maximum sequence length for this model (514 > 512). Running this sequence through the model will result in indexing errors


In [11]:
from keras.preprocessing.sequence import pad_sequences
MAX_LEN = 256
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN , truncating="post", padding="post")

In [12]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, test_size=0.1)


In [13]:
def attention_masking(inputs):
  attention_masks = []
  for sent in inputs:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_masks.append(att_mask)
  return attention_masks

In [14]:
train_masks = attention_masking(train_inputs)
validation_masks = attention_masking(validation_inputs)

In [15]:
import torch

In [16]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)


In [17]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

In [18]:
model = BertForSequenceClassification.from_pretrained(
    "distilroberta-base",
    num_labels = 2,
    output_attentions = False,
    output_hidden_states = False,
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Some weights of the model checkpoint at distilroberta-base were not used when initializing BertForSequenceClassification: ['roberta.embeddings.word_embeddings.weight', 'roberta.embeddings.position_embeddings.weight', 'roberta.embeddings.token_type_embeddings.weight', 'roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.self.query.weight', 'roberta.encoder.layer.0.attention.self.query.bias', 'roberta.encoder.layer.0.attention.self.key.weight', 'roberta.encoder.layer.0.attention.self.key.bias', 'roberta.encoder.layer.0.attention.self.value.weight', 'roberta.encoder.layer.0.attention.self.value.bias', 'roberta.encoder.layer.0.attention.output.dense.weight', 'roberta.encoder.layer.0.attention.output.dense.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.intermediate.dense.weight', 'roberta.encoder.layer.0.intermediate.dense.bias', 

In [19]:
optimizer = AdamW(model.parameters(), lr = 2e-5)

In [20]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

The BERT model has 105 different named parameters.

==== Embedding Layer ====

bert.embeddings.word_embeddings.weight                  (50265, 768)
bert.embeddings.position_embeddings.weight                (514, 768)
bert.embeddings.token_type_embeddings.weight                (1, 768)
bert.embeddings.LayerNorm.weight                              (768,)
bert.embeddings.LayerNorm.bias                                (768,)

==== First Transformer ====

bert.encoder.layer.0.attention.self.query.weight          (768, 768)
bert.encoder.layer.0.attention.self.query.bias                (768,)
bert.encoder.layer.0.attention.self.key.weight            (768, 768)
bert.encoder.layer.0.attention.self.key.bias                  (768,)
bert.encoder.layer.0.attention.self.value.weight          (768, 768)
bert.encoder.layer.0.attention.self.value.bias                (768,)
bert.encoder.layer.0.attention.output.dense.weight        (768, 768)
bert.encoder.layer.0.attention.output.dense.bias              (

In [21]:
from transformers import get_linear_schedule_with_warmup


In [22]:
epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, 
                                            num_training_steps = total_steps)

In [23]:
import numpy as np
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

import time
import datetime

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [24]:
acc_list = []
ep_list = []

In [25]:
import random

seed_val = 57

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

loss_values = []

for epoch_i in range(0, epochs):

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        
        outputs = model(b_input_ids, 
                    token_type_ids=None, 
                    attention_mask=b_input_mask, 
                    labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
    avg_train_loss = total_loss / len(train_dataloader)
    loss_values.append(avg_train_loss)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
    
    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        with torch.no_grad():
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    acc_list.append(eval_accuracy/nb_eval_steps)
    ep_list.append(epoch_i + 1)


print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch    40  of     70.    Elapsed: 0:26:06.

  Average training loss: 0.70
  Training epcoh took: 0:45:12

Running Validation...
  Accuracy: 0.50
  Validation took: 0:01:41

======== Epoch 2 / 4 ========
Training...
  Batch    40  of     70.    Elapsed: 0:26:11.

  Average training loss: 0.69
  Training epcoh took: 0:45:37

Running Validation...
  Accuracy: 0.51
  Validation took: 0:01:37

======== Epoch 3 / 4 ========
Training...
  Batch    40  of     70.    Elapsed: 0:26:25.

  Average training loss: 0.68
  Training epcoh took: 0:45:46

Running Validation...
  Accuracy: 0.52
  Validation took: 0:01:44

======== Epoch 4 / 4 ========
Training...
  Batch    40  of     70.    Elapsed: 0:26:17.

  Average training loss: 0.66
  Training epcoh took: 0:45:50

Running Validation...
  Accuracy: 0.55
  Validation took: 0:01:37

Training complete!


In [26]:
acc_list

[0.5, 0.5065104166666666, 0.5234375, 0.5455729166666666]